In [1]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir_pt
import pyterrier_doc2query
import os
import pandas as pd

PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
rank_cutoff = 30
data = pd.read_csv('document.csv')
data.info()
data['docno'] = data.docno.astype(str)
data = data[['docno', 'description']]
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4266 entries, 0 to 4265
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   docno        4266 non-null   object 
 1   title        4266 non-null   object 
 2   description  4266 non-null   object 
 3   genre        4266 non-null   object 
 4   rating       4266 non-null   float64
 5   num_reviews  4266 non-null   int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 200.1+ KB


,docno,description
0,ug7v899j,In a culture trying to sell us the lie that we...
1,02tnwd4m,Welcome to the Vortex Nation Podcast. Brought ...
2,ejv2xln0,"Greg Laurie delivers compelling, practical ins..."
3,2b73a28n,The Amelia Project is a secret agency that fak...
4,9785vg6d,"Join Rob Zacny, Natalie Watson, Ali Acampora, ..."


In [3]:
indexer = pt.DFIndexer("./pd_base_index", overwrite=True, blocks=True)
index_ref = indexer.index(data["description"], data["docno"],)
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

/Users/yifeizhang/opt/anaconda3/lib/python3.9/site-packages/pyterrier/index.py:621: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, value in meta_column[1].iteritems():


19:39:33.513 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (x80cs5bc) - further warnings are suppressed
19:39:36.333 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents
Number of documents: 4266
Number of terms: 17506
Number of postings: 151825
Number of fields: 0
Number of tokens: 178247
Field names: []
Positions:   true



In [4]:
topics = pd.read_csv('topics.csv') 
topics['qid'] = topics.qid.astype('str')
qrels = pd.read_csv('qrels.csv') 
qrels['qid'] = qrels.qid.astype(str)
qrels['docno'] = qrels.docno.astype(str)


In [5]:
import random
def random_score(keyFreq, posting, entryStats, collStats):
    import numpy as np
    return np.random.randn(1)[0]
random = pt.BatchRetrieve(index, wmodel=random_score)

In [6]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")
pt.Experiment(
    [tf_idf, bm25, random],
    topics,
    qrels,
    eval_metrics=["map", "ndcg","ndcg_cut_5", "ndcg_cut_10"])

,name,map,ndcg,ndcg_cut_5,ndcg_cut_10
0,BR(TF_IDF),0.367018,0.669108,0.702028,0.710234
1,BR(BM25),0.373931,0.671245,0.690518,0.705602
2,BR(DPH),0.165839,0.483323,0.162697,0.170968


In [7]:
# after rankoff, when comparing the new model to bm25 and tfidf, all the models will use the same rank cutoff

pt.Experiment(
    [tf_idf%rank_cutoff, bm25%rank_cutoff, random%rank_cutoff],
    topics,
    qrels,
    eval_metrics=["map", "ndcg","ndcg_cut_5", "ndcg_cut_10"])

,name,map,ndcg,ndcg_cut_5,ndcg_cut_10
0,"RankCutoff(BR(TF_IDF), 30)",0.288072,0.456875,0.702028,0.710234
1,"RankCutoff(BR(BM25), 30)",0.301524,0.470172,0.690518,0.705602
2,"RankCutoff(BR(DPH), 30)",0.038977,0.127632,0.228866,0.208940
